Projeto Final Pós Graduação Big Data - Senac Rio - 2020.

Processo de importação das Bibliotecas utilizadas no durante todo o projeto.

In [1]:
#Importação das bibliotecas utilizadas para manipulação dos dados das paginas Web.

from bs4 import BeautifulSoup
import urllib.request
import shutil
import os
from datetime import datetime
import pandas as pd
import pyspark
import zipfile
import sqlalchemy
import pymysql
import json
import numpy as np
import matplotlib.pyplot as plt

Indicação do diretório para download dos arquivos Inep-Enem. 

In [ ]:
#Indicação do diretório desejado para download dos arquivos. 
#No exemplo, estamos salvando no diretório: C:\Users\Ricardo\Downloads

diretorio = input('Digite o diretório desejado para salvar os arquivos: ')
os.chdir(r'{}'.format(diretorio))
os.getcwd( )

In [ ]:
#Apos indicar o diretorio desejado, como boa pratica, sugerimos a criação da pasta onde será salvos os arquivos.
#No exemplo, estamos salvando na pasta: MicrodadosEnem.'''

url = 'http://inep.gov.br/microdados'

pasta = input('Após indicação do diretório {}, informe o nome da pasta desejada para salvar os arquivos baixados da pagina {}: '.format(diretorio, url))

try:
    os.mkdir(pasta)
    print('Pasta {} criada com sucesso!'.format(pasta))
except FileExistsError as e:
    print(f'Pasta {pasta} já existe')

In [ ]:
#Indicação do diretório onde será salvo os arquivos baixados da url.

os.chdir(r'{}\{}'.format(diretorio, pasta))
print('Os arquivos serão salvos no seguinte diretorio:', format(os.getcwd()))

Processo de Web Scraping no site do Inep, buscando os microdados do Enem.

In [ ]:
#Criação da variável que receberá todos os dados correspondente a varíavel "url".

page = urllib.request.urlopen(url)

In [ ]:
#Criação da variável "soup", do pacote BeatifulSoup, que apresenta a estutura HTML da pagina que será trabalhada.

soup = BeautifulSoup(page,'html.parser')
print(soup)

In [ ]:
#No loop "for" abaixo, o realizaremos o filtro das tags desejadas.

for link in soup.findAll('a'):
    theLink = link.get('href')
    name = link.string
    
    print(theLink)
    print(name)

In [ ]:
#No processo abaixo, realizaremos o filtro das linhas, cujo o o tipo do arquivo desejado é *.zip.

nomeArquivo = []
typeFile = '.zip'
i = 0
for link in soup.findAll('a'):
    theLink = str(link.get('href')).lower()
    name = link.string
    if 'enem'in theLink:
        if theLink[-4:] == typeFile:
            print(theLink)
            print(type(theLink))
            print(name)
            print(type(name))
            nomeArquivo.append(theLink[39:-4].replace('/','_'))
            print(type(nomeArquivo))

In [ ]:
#Processo de download dos arquivos desejados e salvos na na pasta indicada.

i = 0
for link in soup.findAll('a'):
    theLink = str(link.get('href')).lower()
    name = link.string
    if 'enem'in theLink:
        if theLink[-4:] == typeFile:
            if name is None:
                title = 'SemTítulo' + typeFile
            else:
                title = nomeArquivo[i] + typeFile
                i +=1
            try:
                with open(os.getcwd() + '/' + title,'r') as fileExists:
                    print('Arquivo ' + title + ' existente no diretório '+ os.getcwd() + ' - ' + str(datetime.now()))
                    fileExists.close()
            except FileNotFoundError:
                print('Download Iniciado - {}: '.format(datetime.now()) + title)
                docFile = open(os.getcwd() + '/' + title,'wb')
                docFile.write(urllib.request.urlopen(theLink).read())
                docFile.close()
                print('Download Concluído em {}'.format(datetime.now()))
print('Não há mais arquivos para download')

In [ ]:
#Apresentação do tamanho dos arquivos baixados.

print('Apresentação Arquivos Enem baixados no díretório: ' + os.getcwd())
contador = 0
tamanhoTotal = 0
tipo = '.zip'

def formatarTamanho(tamanho):
    base = 1024
    kilo = base
    mega = base ** 2
    giga = base ** 3
    tera = base ** 4
    
    if tamanho < kilo:
        tamanho = base
        texto = 'B'
    elif tamanho < mega:
        tamanho /= kilo
        texto = 'K'
    elif tamanho < giga:
        tamanho /= mega
        texto = 'M'
    else:
        tamanho < tera
        tamanho /= giga
        texto = 'G'
    tamanho = round(tamanho, 2)
    return f'{tamanho} {texto}'

for raiz, diretorios, arquivos in os.walk(os.getcwd()):
    for arquivo in arquivos:
        if tipo in arquivo:
            contador +=1 
            caminho = os.path.join(raiz, arquivo)
            nome, ext = os.path.splitext(arquivo)
            tamanho = os.path.getsize(caminho)
            tamanhoTotal += tamanho
                                
            if tamanho > 0:
                print()
                print('Arquivo encontrado: ', arquivo)
                print('Caminho: ', caminho)
                print('Nome: ', nome)
                print('Extensão: ', ext)
                print('Tamanho: ', formatarTamanho(tamanho))

print()
print(f'{contador} arquivo(s) encontrado(s).' + 'Tamanho Total: ' + formatarTamanho(tamanhoTotal))

Processo de Conexão ao Mysql e criação do banco de dados que receberá os microdados dos anos de 2015 a 2019.

In [ ]:
# Criando um database.

conexao = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = 'admin')

cursor = conexao.cursor()

try:
    cursor.execute('CREATE DATABASE dbenem')
    print('Banco de Dados dbenem criado com sucesso!')
except:
    print('Banco de Dados dbenem já existe!')

In [ ]:
# Conectando ao Banco de Dados enemdb.

cursor.execute("SHOW DATABASES")
print('Databases MySql:')
for x in cursor:
  print(x)


cursor.connection.select_db('dbenem')
print('')
print('Conexão com o banco de dados dbenem')

In [2]:
from pyspark import SparkContext
sc = SparkContext()

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [3]:
# No Processo abaixo, importaremos para o dataframe, apenas para conhecer os dados e verificar o separador do microdado.

print(f'Criação Dataframes Iniciado - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2015 iniciada - {datetime.now()}.')
df_enem2015_completo = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=",", inferschema='true', encoding = 'ISO-8859-1', ).load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\MICRODADOS_ENEM_2015.csv')
print(f'Dataframe df_enem2015_completo criado com sucesso - {datetime.now()}.')
print('')

print(f'Criação Dataframe do Enem 2016 iniciada- {datetime.now()}.')
df_enem2016_completo = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=";", inferschema='true', encoding = 'ISO-8859-1').load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\microdados_enem_2016.csv')
print(f'Dataframe df_enem2016_completo criado com sucesso - {datetime.now()}.')
print('')

print(f'Criação Dataframe do Enem 2017 iniciada - {datetime.now()}.')
df_enem2017_completo = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=";", inferschema='true', encoding = 'ISO-8859-1').load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\MICRODADOS_ENEM_2017.csv')
print(f'Dataframe df_enem2017_completo criado com sucesso - {datetime.now()}.')
print('')

print(f'Criação Dataframe do Enem 2018 iniciada - {datetime.now()}.')
df_enem2018_completo = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=";", inferschema='true', encoding = 'ISO-8859-1').load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\MICRODADOS_ENEM_2018.csv')
print(f'Dataframe df_enem2018_completo criado com sucesso - {datetime.now()}.')
print('')

print(f'Criação Dataframe do Enem 2019 iniciada - {datetime.now()}.')
df_enem2019_completo = sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep=";", inferschema='true', encoding = 'ISO-8859-1').load(r'C:\Users\Ricardo\Downloads\MicrodadosEnem\MICRODADOS_ENEM_2019.csv')
print(f'Dataframe df_enem2019_completo criado com sucesso - {datetime.now()}.')
print('')

print('')
print(f'Dataframes criados com sucesso - {datetime.now()}.')

Criação Dataframes Iniciado - 2020-08-18 09:47:55.725250.


Criação Dataframe do Enem 2015 iniciada - 2020-08-18 09:47:55.725250.
Dataframe df_enem2015_completo criado com sucesso - 2020-08-18 09:48:37.997162.

Criação Dataframe do Enem 2016 iniciada- 2020-08-18 09:48:37.997162.
Dataframe df_enem2016_completo criado com sucesso - 2020-08-18 09:49:28.979376.

Criação Dataframe do Enem 2017 iniciada - 2020-08-18 09:49:28.980374.
Dataframe df_enem2017_completo criado com sucesso - 2020-08-18 09:50:03.062463.

Criação Dataframe do Enem 2018 iniciada - 2020-08-18 09:50:03.062463.
Dataframe df_enem2018_completo criado com sucesso - 2020-08-18 09:50:29.897765.

Criação Dataframe do Enem 2019 iniciada - 2020-08-18 09:50:29.897765.
Dataframe df_enem2019_completo criado com sucesso - 2020-08-18 09:50:54.196779.


Dataframes criados com sucesso - 2020-08-18 09:50:54.197781.


In [ ]:
#Importando dos Dataframes para tabelas no mysql

print(f'Iniciando a criação e importação dos dados do Microdados Enem para tabela mysql - {datetime.now()}.')
print('')

print('')
print(f'Criação da Tabela tb_enem2015 iniciada - {datetime.now()}.')
print(f'Importação do Dataframe enem2015 para a Tabela tb_enem2015 iniciada - {datetime.now()}.')
df_enem2015_completo.write.format('jdbc').options(
    url='jdbc:mysql://localhost/dbenem?useTimezone=true&serverTimezone=UTC',
    driver='com.mysql.jdbc.Driver',
    dbtable='tb_enem2015',
    user='root', password='admin').mode('overwrite').save()
print(f'Importação para a Tabela tb_enem2015 com sucesso - {datetime.now()}.')
print('')

print(f'Criação da Tabela tb_enem2016 iniciada - {datetime.now()}.')
print(f'Importação do Dataframe enem2016 para a Tabela tb_enem2016 iniciada - {datetime.now()}.')
df_enem2016_completo.write.format('jdbc').options(
    url='jdbc:mysql://localhost/dbenem?useTimezone=true&serverTimezone=UTC',
    driver='com.mysql.jdbc.Driver',
    dbtable='tb_enem2016',
    user='root', password='admin').mode('overwrite').save()  
print(f'Importação para a Tabela tb_enem2016 com sucesso - {datetime.now()}.')
print('')

print(f'Criação da Tabela tb_enem2017 iniciada - {datetime.now()}.')
print(f'Importação do Dataframe enem2017 para a Tabela tb_enem2017 iniciada - {datetime.now()}.')
df_enem2017_completo.write.format('jdbc').options(
    url='jdbc:mysql://localhost/dbenem?useTimezone=true&serverTimezone=UTC',
    driver='com.mysql.jdbc.Driver',
    dbtable='tb_enem2017',
    user='root', password='admin').mode('overwrite').save()
print(f'Importação para a Tabela tb_enem2017 com sucesso - {datetime.now()}.')
print('')

print(f'Criação da Tabela tb_enem2018 iniciada - {datetime.now()}.')
print(f'Importação do Dataframe enem2018 para a Tabela tb_enem2018 iniciada - {datetime.now()}.')
df_enem2018_completo.write.format('jdbc').options(
    url='jdbc:mysql://localhost/dbenem?useTimezone=true&serverTimezone=UTC',
    driver='com.mysql.jdbc.Driver',
    dbtable='tb_enem2018',
    user='root', password='admin').mode('overwrite').save()
print(f'Importação para a Tabela tb_enem2018 com sucesso - {datetime.now()}.')
print('')

print(f'Criação da Tabela tb_enem2019 iniciada - {datetime.now()}.')
print(f'Importação do Dataframe enem2019 para a Tabela tb_enem2019 iniciada - {datetime.now()}.')
df_enem2019_completo.write.format('jdbc').options(
    url='jdbc:mysql://localhost/dbenem?useTimezone=true&serverTimezone=UTC',
    driver='com.mysql.jdbc.Driver',
    dbtable='tb_enem2019',
    user='root', password='admin').mode('overwrite').save()    
print(f'Importação para a Tabela tb_enem2019 com sucesso - {datetime.now()}.')
print('')

print('')
print(f'Tabelas e Importações realizados com sucesso - {datetime.now()}.')

In [4]:
print('Quantidade de Registros importadados para cada Dataframe')
print('')
print('Quantdade de Registros Dataframe Enem2015')
print(df_enem2015_completo.count())
print('')
print('Quantdade de Registros Dataframe Enem2016')
print(df_enem2016_completo.count())
print('')
print('Quantdade de Registros Dataframe Enem2017')
print(df_enem2017_completo.count())
print('')
print('Quantdade de Registros Dataframe Enem2018')
print(df_enem2018_completo.count())
print('')
print('Quantdade de Registros Dataframe Enem2019')
print(df_enem2019_completo.count())

Quantidade de Registros importadados para cada Dataframe

Quantdade de Registros Dataframe Enem2015
7746427

Quantdade de Registros Dataframe Enem2016
8627367

Quantdade de Registros Dataframe Enem2017
6731341

Quantdade de Registros Dataframe Enem2018
5513747

Quantdade de Registros Dataframe Enem2019
5095270


In [8]:
df_enem2015_completo.show(n=2,truncate=True)

+------------+------+-----------------------+-----------------------+----------------+----------------+---------------------------+------------+---------+----------------+----------------+---------+---------+----------------------+------------------+---------------+--------+-------+----------------+-----------------------+-----------------------+----------------+----------------+---------------+---------------+---------+---------+---------------+-----------+--------------+-----------+---------+-----------------------+-----------------+---------------------+---------------------+------------------+-----------+-----------+-----------+--------+--------------+----------+------------------+------------+------------+--------------+--------------+----------+--------------+--------------+--------+---------+--------------+---------+-----------------+---------------------+------------------------+--------------+------------------+-------+-------------+-------------------+------------------+-----

In [9]:
df_enem2015_completo.head(2)

[Row(NU_INSCRICAO=150000000376, NU_ANO=2015, CO_MUNICIPIO_RESIDENCIA=4309209, NO_MUNICIPIO_RESIDENCIA='Gravataí', CO_UF_RESIDENCIA=43, SG_UF_RESIDENCIA='RS', IN_ESTUDA_CLASSE_HOSPITALAR=0, IN_TREINEIRO=0, CO_ESCOLA=None, CO_MUNICIPIO_ESC=None, NO_MUNICIPIO_ESC=None, CO_UF_ESC=None, SG_UF_ESC=None, TP_DEPENDENCIA_ADM_ESC=None, TP_LOCALIZACAO_ESC=None, TP_SIT_FUNC_ESC=None, NU_IDADE=42, TP_SEXO='M', TP_NACIONALIDADE=1, CO_MUNICIPIO_NASCIMENTO=4209003, NO_MUNICIPIO_NASCIMENTO='Joaçaba', CO_UF_NASCIMENTO=42, SG_UF_NASCIMENTO='SC', TP_ST_CONCLUSAO=1, TP_ANO_CONCLUIU=10, TP_ESCOLA=1, TP_ENSINO=None, TP_ESTADO_CIVIL=1, TP_COR_RACA=1, IN_BAIXA_VISAO=0, IN_CEGUEIRA=0, IN_SURDEZ=0, IN_DEFICIENCIA_AUDITIVA=0, IN_SURDO_CEGUEIRA=0, IN_DEFICIENCIA_FISICA=0, IN_DEFICIENCIA_MENTAL=0, IN_DEFICIT_ATENCAO=0, IN_DISLEXIA=0, IN_GESTANTE=0, IN_LACTANTE=0, IN_IDOSO=0, IN_DISCALCULIA=0, IN_AUTISMO=0, IN_VISAO_MONOCULAR=0, IN_SABATISTA=0, IN_OUTRA_DEF=0, IN_SEM_RECURSO=0, IN_NOME_SOCIAL=0, IN_BRAILLE=0, IN_AMP

In [10]:
df_enem2015_completo.printSchema()

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- CO_MUNICIPIO_RESIDENCIA: integer (nullable = true)
 |-- NO_MUNICIPIO_RESIDENCIA: string (nullable = true)
 |-- CO_UF_RESIDENCIA: integer (nullable = true)
 |-- SG_UF_RESIDENCIA: string (nullable = true)
 |-- IN_ESTUDA_CLASSE_HOSPITALAR: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_ESCOLA: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- NU_IDADE: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- CO_MUNICIPIO_NASCIMENTO: integer (nullable = true)
 |-- NO_MUNICIPIO_NASCIMENTO: string (nu

In [11]:
df_enem2015_completo.columns[:]

['NU_INSCRICAO',
 'NU_ANO',
 'CO_MUNICIPIO_RESIDENCIA',
 'NO_MUNICIPIO_RESIDENCIA',
 'CO_UF_RESIDENCIA',
 'SG_UF_RESIDENCIA',
 'IN_ESTUDA_CLASSE_HOSPITALAR',
 'IN_TREINEIRO',
 'CO_ESCOLA',
 'CO_MUNICIPIO_ESC',
 'NO_MUNICIPIO_ESC',
 'CO_UF_ESC',
 'SG_UF_ESC',
 'TP_DEPENDENCIA_ADM_ESC',
 'TP_LOCALIZACAO_ESC',
 'TP_SIT_FUNC_ESC',
 'NU_IDADE',
 'TP_SEXO',
 'TP_NACIONALIDADE',
 'CO_MUNICIPIO_NASCIMENTO',
 'NO_MUNICIPIO_NASCIMENTO',
 'CO_UF_NASCIMENTO',
 'SG_UF_NASCIMENTO',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'TP_ENSINO',
 'TP_ESTADO_CIVIL',
 'TP_COR_RACA',
 'IN_BAIXA_VISAO',
 'IN_CEGUEIRA',
 'IN_SURDEZ',
 'IN_DEFICIENCIA_AUDITIVA',
 'IN_SURDO_CEGUEIRA',
 'IN_DEFICIENCIA_FISICA',
 'IN_DEFICIENCIA_MENTAL',
 'IN_DEFICIT_ATENCAO',
 'IN_DISLEXIA',
 'IN_GESTANTE',
 'IN_LACTANTE',
 'IN_IDOSO',
 'IN_DISCALCULIA',
 'IN_AUTISMO',
 'IN_VISAO_MONOCULAR',
 'IN_SABATISTA',
 'IN_OUTRA_DEF',
 'IN_SEM_RECURSO',
 'IN_NOME_SOCIAL',
 'IN_BRAILLE',
 'IN_AMPLIADA_24',
 'IN_AMPLIADA_18',
 'IN_L

In [12]:
df_enem2015_completo.describe('NU_IDADE').show()

+-------+------------------+
|summary|          NU_IDADE|
+-------+------------------+
|  count|           7746352|
|   mean|23.004694338702915|
| stddev| 7.944599707765686|
|    min|                10|
|    max|                98|
+-------+------------------+



In [13]:
df_enem2015_completo.describe('NU_NOTA_CN').show()

+-------+-----------------+
|summary|       NU_NOTA_CN|
+-------+-----------------+
|  count|          5755091|
|   mean|478.9239503771539|
| stddev|71.56005864237716|
|    min|              0.0|
|    max|            875.2|
+-------+-----------------+



In [14]:
df_enem2015_completo.describe('NU_NOTA_CH').show()

+-------+-----------------+
|summary|       NU_NOTA_CH|
+-------+-----------------+
|  count|          5755091|
|   mean|558.4878240152929|
| stddev|70.96839011437301|
|    min|              0.0|
|    max|            850.6|
+-------+-----------------+



In [15]:
df_enem2015_completo.describe('NU_NOTA_LC').show()

+-------+------------------+
|summary|        NU_NOTA_LC|
+-------+------------------+
|  count|           5625540|
|   mean|505.45478531838677|
| stddev| 72.68930504276108|
|    min|               0.0|
|    max|             825.8|
+-------+------------------+



In [16]:
df_enem2015_completo.describe('NU_NOTA_MT').show()

+-------+------------------+
|summary|        NU_NOTA_MT|
+-------+------------------+
|  count|           5625540|
|   mean|467.93117604709937|
| stddev|107.36792743837316|
|    min|               0.0|
|    max|            1008.3|
+-------+------------------+



In [17]:
df_enem2015_completo.describe('NU_NOTA_REDACAO').show()

+-------+------------------+
|summary|   NU_NOTA_REDACAO|
+-------+------------------+
|  count|           7744736|
|   mean|388.40985670783357|
| stddev| 267.5071259318306|
|    min|                 0|
|    max|              1000|
+-------+------------------+



In [26]:
columns = [
'NU_INSCRICAO', 
'NU_ANO', 
'NO_MUNICIPIO_RESIDENCIA', 
'CO_UF_RESIDENCIA', 
'SG_UF_RESIDENCIA',
'NU_IDADE', 
'TP_SEXO', 
'TP_ESTADO_CIVIL',
'TP_COR_RACA', 
'TP_NACIONALIDADE',
'NO_MUNICIPIO_NASCIMENTO',
'CO_UF_NASCIMENTO', 
'SG_UF_NASCIMENTO', 
'TP_ST_CONCLUSAO',
'TP_ESCOLA', 
'TP_ENSINO', 
'IN_TREINEIRO', 
'CO_UF_ESC', 
'SG_UF_ESC',
'TP_DEPENDENCIA_ADM_ESC', 
'TP_LOCALIZACAO_ESC', 
'IN_BAIXA_VISAO', 
'IN_CEGUEIRA', 
'IN_SURDEZ',
'IN_DEFICIENCIA_AUDITIVA', 
'IN_SURDO_CEGUEIRA',
'IN_DEFICIENCIA_FISICA', 
'IN_DEFICIENCIA_MENTAL',
'IN_DEFICIT_ATENCAO', 
'IN_DISLEXIA', 
'IN_DISCALCULIA', 
'IN_AUTISMO', 
'IN_VISAO_MONOCULAR',
'IN_OUTRA_DEF', 
'IN_GESTANTE', 
'IN_LACTANTE',
'IN_IDOSO', 
'IN_ESTUDA_CLASSE_HOSPITALAR', 
'IN_SEM_RECURSO', 
'IN_BRAILLE', 
'IN_AMPLIADA_24', 
'IN_AMPLIADA_18', 
'IN_LEDOR',
'IN_ACESSO', 
'IN_TRANSCRICAO', 
'IN_LIBRAS', 
'IN_LEITURA_LABIAL',
'IN_MESA_CADEIRA_RODAS', 
'IN_MESA_CADEIRA_SEPARADA',
'IN_APOIO_PERNA', 
'IN_GUIA_INTERPRETE', 
'IN_COMPUTADOR',
'IN_CADEIRA_ESPECIAL', 
'IN_CADEIRA_CANHOTO',
'IN_CADEIRA_ACOLCHOADA', 
'IN_PROVA_DEITADO', 
'IN_MOBILIARIO_OBESO',
'IN_LAMINA_OVERLAY', 
'IN_PROTETOR_AURICULAR', 
'IN_MEDIDOR_GLICOSE',
'IN_MAQUINA_BRAILE', 
'IN_SOROBAN', 
'IN_MARCA_PASSO', 
'IN_SONDA',
'IN_MEDICAMENTOS', 
'IN_SALA_INDIVIDUAL', 
'IN_SALA_ESPECIAL',
'IN_SALA_ACOMPANHANTE', 
'IN_MOBILIARIO_ESPECIFICO',
'IN_NOME_SOCIAL',
'NO_MUNICIPIO_PROVA', 
'CO_UF_PROVA', 
'SG_UF_PROVA',
'TP_PRESENCA_CN', 
'TP_PRESENCA_CH', 
'TP_PRESENCA_LC',
'TP_PRESENCA_MT', 
'NU_NOTA_CN', 
'NU_NOTA_CH', 
'NU_NOTA_LC',
'NU_NOTA_MT', 
'TP_LINGUA',
'TP_STATUS_REDACAO', 
'NU_NOTA_COMP1',
'NU_NOTA_COMP2', 
'NU_NOTA_COMP3', 
'NU_NOTA_COMP4', 
'NU_NOTA_COMP5',
'NU_NOTA_REDACAO', 
'Q001', 
'Q002', 
'Q005', 
'Q006',
'Q022',
'Q024', 
'Q025'
]


In [27]:
# No Processo abaixo, criaremos dataframes, apenas com colunas selecionadas.

print(f'Criação Dataframes Iniciado - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2015 iniciada - {datetime.now()}.')
df_enem2015 = df_enem2015_completo.select(columns)
print(f'Dataframe df_enem2015 criado com sucesso - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2016 iniciada - {datetime.now()}.')
df_enem2016 = df_enem2016_completo.select(columns)
print(f'Dataframe df_enem2016 criado com sucesso - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2017 iniciada - {datetime.now()}.')
df_enem2017 = df_enem2017_completo.select(columns)
print(f'Dataframe df_enem2017 criado com sucesso - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2018 iniciada - {datetime.now()}.')
df_enem2018 = df_enem2018_completo.select(columns)
print(f'Dataframe df_enem2018 criado com sucesso - {datetime.now()}.')
print('')

print('')
print(f'Criação Dataframe do Enem 2019 iniciada - {datetime.now()}.')
df_enem2019 = df_enem2019_completo.select(columns)
print(f'Dataframe df_enem2019 criado com sucesso - {datetime.now()}.')
print('')


Criação Dataframes Iniciado - 2020-08-18 10:29:40.608216.


Criação Dataframe do Enem 2015 iniciada - 2020-08-18 10:29:40.608216.
Dataframe df_enem2015 criado com sucesso - 2020-08-18 10:29:40.730054.


Criação Dataframe do Enem 2016 iniciada - 2020-08-18 10:29:40.731035.
Dataframe df_enem2016 criado com sucesso - 2020-08-18 10:29:40.837013.


Criação Dataframe do Enem 2017 iniciada - 2020-08-18 10:29:40.837013.
Dataframe df_enem2017 criado com sucesso - 2020-08-18 10:29:40.972231.


Criação Dataframe do Enem 2018 iniciada - 2020-08-18 10:29:40.973195.
Dataframe df_enem2018 criado com sucesso - 2020-08-18 10:29:41.109827.


Criação Dataframe do Enem 2019 iniciada - 2020-08-18 10:29:41.110827.
Dataframe df_enem2019 criado com sucesso - 2020-08-18 10:29:41.204575.



In [36]:
df_enem = df_enem2015.unionAll(df_enem2016).unionAll(df_enem2017).unionAll(df_enem2018).unionAll(df_enem2019)

In [37]:
df_enem.count()

33714152